# California Recovered Ghost Guns

The public data set, `a_ghost_guns_by_county_monthly.csv`, last updated July 2025, was acquired from the California Department of Justice data provided by the Gun Violence Data Hub.



In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import geopandas as gpd
from os import name

from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import numpy as np

# Reformating the Data Frame

In [ ]:
og = pd.read_csv("/content/ca_ghost_guns_by_county_monthly.csv")

# CHANGE TYPE from date to date-time
og["year_month"] = pd.to_datetime(og["year_month"])


### Add column for percentage of ghost guns recovered, and make a quarterly and yearly level of the `og`

In [ ]:
# MONTHLY - ca_monthly (2010-2025)
ca_monthly = og.groupby(["year", "year_month"], as_index=False).agg({"total_crime_guns":"sum", "ghost_guns":"sum"})
ca_monthly["pct_gg_recovered"] = ((ca_monthly["ghost_guns"]/ca_monthly["total_crime_guns"])*100).round(2)

# YEARLY Total Crimes and Ghost Guns - ca_yearly (2010-2024)
ca_yearly = og.groupby("year", as_index=False).agg({"total_crime_guns":"sum", "ghost_guns":"sum"})
ca_yearly = ca_yearly[(ca_yearly["year"] < 2025)]
ca_yearly["pct_gg_recovered"] = ((ca_yearly["ghost_guns"]/ca_yearly["total_crime_guns"])*100).round(2)


### Add Ghost Guns per 100k on a County Level for `ca_yearly`

Adding a Ghost Guns Recovered per 100,000 Population column to the `og` df.

Rateper100,000=(NumberofCases/PopulationSize)∗100,000

In [ ]:
# GROUPBY County, Year and Population, TOT of Gun Related Crims and Ghost Guns Recovered
county = og.groupby(["county", "year", "population"], as_index=False).agg({"total_crime_guns":"sum", "ghost_guns":"sum"})
county = county[(county["year"] < 2025)]

# MAKE NEW COLUMN - Ghost Guns per 100k
county["ghost_guns_per_100k"] = (county["ghost_guns"] / county["population"]) * 100000
county["ghost_guns_per_100k"] = county["ghost_guns_per_100k"].round(2)

### Export all Data Frames

In [ ]:
# EXPORT
# except og
ca_monthly.to_csv("ca_monthly.csv", index=False)
ca_yearly.to_csv("ca_yearly.csv", index=False)
county.to_csv("county.csv", index=False)

# Graphs

### Monthly Subplots

In [ ]:
# make_subplot()
subplot_monthly = make_subplots(rows=2, cols=1,
                                  shared_xaxes=True,
                                  vertical_spacing=0.15,
                                  subplot_titles=("Crime Guns Recovered", "Ghost Guns Recovered"),
                                  x_title="Year",
                                  y_title="Total Recovered"
                                  )

# ADD CRIME GUNS to subplot
subplot_monthly.add_trace(go.Scatter(x=ca_monthly["year_month"],
                                       y=ca_monthly["total_crime_guns"],
                                       name="Crime Guns",
                                       mode="lines",
                                       showlegend=False,
                                       xaxis="x"
                                       ),

                          row=1, col=1
                          )

# ADD GHOST GUNS to subplot
subplot_monthly.add_trace(go.Scatter(x=ca_monthly["year_month"],
                                       y=ca_monthly["ghost_guns"],
                                       name="Ghost Guns",
                                       mode="lines",
                                       showlegend=False,
                                       xaxis="x"
                                       ),
                          row=2, col=1
                          )

# UPDATE LAYOUT
subplot_monthly.update_layout(title_text="Crimes Guns and Ghost Guns Recovered Monthly in California",
                                title_subtitle_text="From 2010 to 2025",
                                plot_bgcolor="white",
                                hoverlabel=dict(bgcolor="white",
                                                bordercolor="black",
                                                font_color="black"
                                                ),
                                hoversubplots="axis",
                                hovermode="x unified",
                                width=700,
                                height=700
                                )

subplot_monthly.update_xaxes(showline=True,
                              linewidth=1,
                              linecolor="black",
                              gridcolor="lightgray",
                              griddash="solid",
                              )

subplot_monthly.update_yaxes(showline=True,
                     linewidth=1,
                     linecolor="black",
                     gridcolor="lightgray",
                     griddash="solid"
                     )

subplot_monthly.show()

### Yearly Subplots

In [ ]:
# make_subplot()
subplot_yearly = make_subplots(rows=2, cols=1,
                                  shared_xaxes=True,
                                  vertical_spacing=0.15,
                                  subplot_titles=("Crime Guns Recovered", "Ghost Guns Recovered"),
                                  x_title="Year",
                                  y_title="Total Recovered"
                                  )

# ADD CRIME GUNS to subplot
subplot_yearly.add_trace(go.Scatter(x=ca_yearly["year"],
                                       y=ca_yearly["total_crime_guns"],
                                       name="Crime Guns",
                                       mode="lines+markers",
                                       showlegend=False,
                                       xaxis="x"
                                       ),

                          row=1, col=1
                          )

# ADD GHOST GUNS to subplot
subplot_yearly.add_trace(go.Scatter(x=ca_yearly["year"],
                                       y=ca_yearly["ghost_guns"],
                                       name="Ghost Guns",
                                       mode="lines+markers",
                                       showlegend=False,
                                       xaxis="x"
                                       ),
                          row=2, col=1
                          )

# UPDATE LAYOUT
subplot_yearly.update_layout(title_text="Crimes Guns and Ghost Guns Recovered Yearly in California",
                                title_subtitle_text="From 2010 to 2025",
                                plot_bgcolor="white",
                                hoverlabel=dict(bgcolor="white",
                                                bordercolor="black",
                                                font_color="black"
                                                ),
                                hoversubplots="axis",
                                hovermode="x unified",
                                width=700,
                                height=700
                                )

subplot_yearly.update_xaxes(showline=True,
                            linewidth=1,
                            linecolor="black",
                            gridcolor="lightgray",
                            griddash="solid",
                            range=[2010,2025]
                              )

subplot_yearly.update_yaxes(showline=True,
                     linewidth=1,
                     linecolor="black",
                     gridcolor="lightgray",
                     griddash="solid"
                     )

subplot_yearly.show()

## Choropleth Map - County Level, Yearly

The geoJSON file of California Counties was acquired from [State of California Open Data](https://lab.data.ca.gov/dataset/california-counties2).

In [ ]:
# READ geoJSON file with geopandas
map = gpd.read_file("/content/California_Counties.geojson")

# FIX the "NAME" column of map by removing "county" from the ends of county names
map["NAME"] = map["NAME"].str.replace(" County","")

# MERGE county with map
final_county = map.merge(county, left_on="NAME", right_on="county", how="left")

# CONVERT geopandas read file to GeoJSON for plotly
map_json = json.loads(map.to_json())

In [ ]:
#### PLOT choropleth

# Add population to the hover data

fig5 = px.choropleth(
    final_county,
    geojson=map_json,
    locations="NAME", # not indexed
    featureidkey="properties.NAME",
    color="ghost_guns_per_100k",
    animation_frame="year",
    animation_group="county",
    range_color=[0, 80],
    color_continuous_scale="ylorrd",
    projection="mercator",
    fitbounds="locations",
    basemap_visible=False,
    labels={
        "year":"Year",
        "NAME":"County",
        "ghost_guns_per_100k":"Ghost Guns per 100k"
        }
    )

# AESTHETICS
fig5.update_layout(
    title_text="Ghost Guns per 100,000 People",
    title_subtitle_text="Recovered Ghost Guns per 100,000 People in California Counties Each Year",
    legend_title_side="top center"

)

fig5.show()

# Statistical Analysis - WIP

1. Ratio / Proportion Analysis
2. Regression Discontinuity

### Ratio / Proportional Anlysis

In [ ]:
ratio_quarterly = px.line(ca_monthly, x="year_month",
                          y="pct_gg_recovered",
                          title="Percentage Recovered Ghost Guns to Crime Guns",
                          labels={"pct_gg_recovered":"Percentage of Ghost Guns (%)",
                                  "quarter": "Quarter ",
                                  "quarter":"Year (Quarters)"
                                  }
                          )

ratio_quarterly.update_layout(title_subtitle_text="From 2010 to 2025",
                              plot_bgcolor="white",
                              hovermode="x unified"
                              )
ratio_quarterly.update_xaxes(griddash="solid",
                             gridcolor="lightgray",
                             linecolor="black"
                            )
ratio_quarterly.update_yaxes(gridcolor="lightgray",
                             griddash="solid",
                             linecolor="black"
                             )

ratio_quarterly.show()

### Regression Discontinuity without Total Crime Guns and Linear Regression

In [ ]:
# REGRESSION DISCONTINUITY

## NUMERIC TIME VARIABLE
analysis = ca_monthly[ca_monthly["year_month"] >= "2021-01-01"].copy()
analysis["month_index"] = (analysis["year_month"].dt.year - 2021) * 12 + analysis["year_month"].dt.month

policy = pd.to_datetime("2022-08-01")
policy_index = analysis.loc[analysis["year_month"] == policy, "month_index"].values[0]

## TREATMENT INDICATOR
analysis["post"] = (analysis["month_index"] > policy_index).astype(int)

## RUNNING VARIABLE
analysis["running"] = analysis["month_index"] - policy_index

X = pd.DataFrame({"running": analysis["running"],
                  "post": analysis["post"],
                  "interaction": analysis["running"] * analysis["post"],
                })
y = analysis["ghost_guns"]

model = LinearRegression()
model.fit(X, y)

print("Coefficients:")
for name, coef in zip(X.columns, model.coef_):
    print(f"{name}: {coef:.3f}")
print(f"Intercept: {model.intercept_:.3f}")

## Predicted values for plotting
analysis['pred'] = model.predict(X)

## Pre/Post Policy with Ghost Guns
pre = analysis[analysis["year_month"] <= "2022-08-01"].copy()
post = analysis[analysis["year_month"] > "2022-08-01"].copy()

In [ ]:
# LINEAR REGRESSION for Total Crime Guns

analysis_totgun = ca_monthly[ca_monthly["year_month"] >= "2021-01-01"].copy()
analysis_totgun['month_index'] = (analysis_totgun['year_month'].dt.year - 2021) * 12 + (analysis_totgun['year_month'].dt.month - 1)

X = analysis_totgun["month_index"].values.reshape(-1, 1) # Reshape X to be 2-dimensional (WHY)
y = analysis_totgun["total_crime_guns"]

model = LinearRegression()
model.fit(X, y)

print(f"Intercept: {model.intercept_:.3f}")
print(f"Slope: {model.coef_[0]:.3f}")

x_range = np.linspace(analysis_totgun['month_index'].min(), analysis_totgun['month_index'].max(), 200).reshape(-1, 1)
y_pred = model.predict(x_range)

# Calculate the date range by adding the equivalent number of days
start_date = pd.to_datetime('2021-01-01')
date_range = start_date + pd.to_timedelta(x_range.flatten() * 30.44, unit='D') # Approximate number of days in a month

In [ ]:
# Graphing
subplot_analysis = make_subplots(rows=2, cols=1,
                                 shared_xaxes=True,
                                 vertical_spacing=0.1,
                                 subplot_titles=("Crime Guns Recovered", "Ghost Guns Recovered"),
                                 x_title="Year (Monthly)",
                                 y_title="Total Recovered"
                                )

## Crime Guns
subplot_analysis.add_trace(go.Scatter(x=ca_monthly["year_month"],
                                      y=ca_monthly["total_crime_guns"],
                                      name="Crime Guns",
                                      mode="lines",
                                      showlegend=False,
                                      xaxis="x",
                                      line_color="black"
                                      ),
                          row=1, col=1)
### Linear Regression
subplot_analysis.add_trace(go.Scatter(x=date_range,
                                      y=y_pred,
                                      mode='lines',
                                      name='Linear Fit',
                                      line_color="Red",
                                      hovertemplate = "slope: -17.879"
                                      ),
                           row=1, col=1
                          )

# Ghost Guns
subplot_analysis.add_trace(go.Scatter(x=ca_monthly["year_month"],
                                      y=ca_monthly["ghost_guns"],
                                      name="Ghost Guns",
                                      mode="lines",
                                      showlegend=False,
                                      line_color="black"
                                      ),
                           row=2, col=1
                          )

### Regression Discontinuity
subplot_analysis.add_trace(go.Scatter(x=pre['year_month'],
                                      y=pre['pred'],
                                      mode='lines',
                                      name='Pre-Policy Fitted (RD)',
                                      line=dict(color='Tomato',width=2),
                                      hovertemplate = "slope: -0.070"
                                     ),
                           row=2, col=1
                          )
subplot_analysis.add_trace(go.Scatter(x=post['year_month'],
                                      y=post['pred'],
                                      mode='lines',
                                      name='Post-Policy Fitted (RD)',
                                      hovertemplate = "slope: -102.225",
                                      line=dict(color='RoyalBlue',width=2),
                                     ),
                           row=2, col=1
                          )


# Aesthetics
subplot_analysis.add_vline(x=pd.to_datetime("08-01-2022"),
                           line_width=1, line_dash="dot",
                           line_color="black",
                           )

subplot_analysis.update_layout(title_text="Crimes Guns and Ghost Guns Recovered Monthly in California",
                                title_subtitle_text="From 2010 to 2025",
                                plot_bgcolor="White",
                                hoverlabel=dict(bgcolor="white",
                                                bordercolor="black",
                                                font_color="black"
                                                ),
                                hoversubplots="single",
                                hovermode="x unified",
                                width=1000,
                                height=800
                                )

subplot_analysis.update_xaxes(showline=True,
                              linewidth=1,
                              linecolor="Black",
                              gridcolor="MistyRose",
                              griddash="solid",
                              )

subplot_analysis.update_yaxes(showline=True,
                     linewidth=1,
                     linecolor="Black",
                     gridcolor="MistyRose",
                     griddash="solid"
                     )


### Regression Discontinuity without Total Crime Guns

In [ ]:
# REGRESSION DISCONTINUITY with scikitlearn

## NUMERIC TIME VARIABLE
analysis = ca_monthly[ca_monthly["year_month"] >= "2021-01-01"].copy()
analysis["month_index"] = (analysis["year_month"].dt.year - 2021) * 12 + analysis["year_month"].dt.month

policy = pd.to_datetime("2022-08-01")
policy_index = analysis.loc[analysis["year_month"] == policy, "month_index"].values[0]

## TREATMENT INDICATOR
analysis["post"] = (analysis["month_index"] > policy_index).astype(int)

## RUNNING VARIABLE
analysis["running"] = analysis["month_index"] - policy_index

X = pd.DataFrame({"running": analysis["running"],
                  "post": analysis["post"],
                  "interaction": analysis["running"] * analysis["post"],
                })
y = analysis["ghost_guns"]


## Add constant for intercept
X = sm.add_constant(X)

## Fit OLS model
ols_model = sm.OLS(y, X).fit()

## Print full regression summary (includes p-values & SE)
print(ols_model.summary())

## Predicted values for plotting
analysis['pred'] = ols_model.predict(X)

## Pre/Post Policy with Ghost Guns
pre = analysis[analysis["year_month"] <= "2022-08-01"].copy()
post = analysis[analysis["year_month"] > "2022-08-01"].copy()



In [ ]:
ols_model_robust = ols_model.get_robustcov_results(cov_type='HC1')
print(ols_model_robust.summary())

In [ ]:
# Graphing
subplot_analysis = go.Figure()
# Ghost Guns
subplot_analysis.add_trace(go.Scatter(x=ca_monthly["year_month"],
                                      y=ca_monthly["ghost_guns"],
                                      name="Ghost Guns",
                                      mode="lines",
                                      showlegend=False,
                                      line_color="black"
                                      )
                          )

### Regression Discontinuity
subplot_analysis.add_trace(go.Scatter(x=pre['year_month'],
                                      y=pre['pred'],
                                      mode='lines',
                                      name='Pre-Policy Fitted (RD)',
                                      line=dict(color='Tomato',width=2),
                                      #hovertemplate = "slope: -0.070"
                                     )
                          )
subplot_analysis.add_trace(go.Scatter(x=post['year_month'],
                                      y=post['pred'],
                                      mode='lines',
                                      name='Post-Policy Fitted (RD)',
                                      #hovertemplate = "slope: -102.225",
                                      line=dict(color='RoyalBlue',width=2),
                                     )
                          )

# Aesthetics
subplot_analysis.add_vline(x=pd.to_datetime("08-01-2022"),
                           line_width=1, line_dash="dot",
                           line_color="black",
                           )

subplot_analysis.update_layout(title_text="Crimes Guns and Ghost Guns Recovered Monthly in California",
                                title_subtitle_text="From 2010 to 2025",
                                plot_bgcolor="White",
                                hoverlabel=dict(bgcolor="white",
                                                bordercolor="black",
                                                font_color="black"
                                                ),
                                #hoversubplots="single",
                                hovermode="x unified",
                                )

subplot_analysis.update_xaxes(title="Time (Monthly)",
                              showline=True,
                              linewidth=1,
                              linecolor="Black",
                              gridcolor="MistyRose",
                              griddash="solid",
                              )

subplot_analysis.update_yaxes(title="Total Recovered Ghost Guns",
                              showline=True,
                              linewidth=1,
                              linecolor="Black",
                              gridcolor="MistyRose",
                              griddash="solid"
                              )


### Regression Discontinuity with Total Crime Guns

In [ ]:
# REGRESSION DISCONTINUITY with scikitlearn

## NUMERIC TIME VARIABLE
analysis = ca_monthly[ca_monthly["year_month"] >= "2021-01-01"].copy()
analysis["month_index"] = (analysis["year_month"].dt.year - 2021) * 12 + analysis["year_month"].dt.month

policy = pd.to_datetime("2022-08-01")
policy_index = analysis.loc[analysis["year_month"] == policy, "month_index"].values[0]

## TREATMENT INDICATOR
analysis["post"] = (analysis["month_index"] > policy_index).astype(int)

## RUNNING VARIABLE
analysis["running"] = analysis["month_index"] - policy_index

X = pd.DataFrame({"running": analysis["running"],
                  "post": analysis["post"],
                  "interaction": analysis["running"] * analysis["post"],
                  "total_crime_guns": analysis["total_crime_guns"]
                })
y = analysis["ghost_guns"]


## Add constant for intercept
X = sm.add_constant(X)

## Fit OLS model
ols_model = sm.OLS(y, X).fit()

## Print full regression summary (includes p-values & SE)
print(ols_model.summary())

## Predicted values for plotting
analysis['pred'] = ols_model.predict(X)

## Pre/Post Policy with Ghost Guns
pre = analysis[analysis["year_month"] <= "2022-08-01"].copy()
post = analysis[analysis["year_month"] > "2022-08-01"].copy()

running_pre = np.linspace(pre["running"].min(), 0, 50)
running_post = np.linspace(0, post["running"].max(), 50)

# Map running variable back to dates
dates_pre = policy + pd.to_timedelta(running_pre * 30, unit='D')  # approximate month
dates_post = policy + pd.to_timedelta(running_post * 30, unit='D')

params = ols_model.params

# Predicted values using mean of control variable
mean_pre_control = pre["total_crime_guns"].mean()
mean_post_control = post["total_crime_guns"].mean()

pred_pre = params["const"] + params["running"] * running_pre + params["total_crime_guns"] * mean_pre_control
pred_post = (params["const"] + params["post"] +
             (params["running"] + params["interaction"]) * running_post +
             params["total_crime_guns"] * mean_post_control)


In [ ]:
ols_model_robust = ols_model.get_robustcov_results(cov_type='HC1')
print(ols_model_robust.summary())

In [ ]:
# Graphing
subplot_analysis = go.Figure()

## Ghost Guns
subplot_analysis.add_trace(go.Scatter(x=ca_monthly["year_month"],
                                      y=ca_monthly["ghost_guns"],
                                      name="Ghost Guns",
                                      mode="lines",
                                      showlegend=False,
                                      line_color="black"
                                      )
                          )

### Regression Discontinuity
subplot_analysis.add_trace(go.Scatter(x=dates_pre,
                                      y=pred_pre,
                                      mode='lines',
                                      name='Pre-Policy Fitted (RD)',
                                      line=dict(color='Tomato',width=2),
                                     )
                          )
subplot_analysis.add_trace(go.Scatter(x=dates_post,
                                      y=pred_post,
                                      mode='lines',
                                      name='Post-Policy Fitted (RD)',
                                      line=dict(color='RoyalBlue',width=2),
                                     )
                          )


# Aesthetics
subplot_analysis.add_vline(x=pd.to_datetime("08-01-2022"),
                           line_width=1, line_dash="dot",
                           line_color="black",
                           )

subplot_analysis.update_layout(title_text="Crimes Guns and Ghost Guns Recovered Monthly in California",
                                title_subtitle_text="From 2010 to 2025",
                                plot_bgcolor="White",
                                hoverlabel=dict(bgcolor="white",
                                                bordercolor="black",
                                                font_color="black"
                                                ),
                                hovermode="x unified",
                                )

subplot_analysis.update_xaxes(title="Time (Monthly)",
                              showline=True,
                              linewidth=1,
                              linecolor="Black",
                              gridcolor="MistyRose",
                              griddash="solid",
                              )

subplot_analysis.update_yaxes(title="Total Recovered Ghost Guns",
                              showline=True,
                              linewidth=1,
                              linecolor="Black",
                              gridcolor="MistyRose",
                              griddash="solid"
                              )
